In [1]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from data import get_prices
from plotly.subplots import make_subplots
from typing import Any

In [3]:
get_prices(4, 2)

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,2,0,COCONUT,10135.0,150.0,10134.0,64.0,NaN,NaN,10136,150,10137.0,64.0,NaN,NaN,10135.5,0.0
1,2,0,COCONUT_COUPON,682.0,38.0,NaN,NaN,NaN,NaN,683,38,NaN,NaN,NaN,NaN,682.5,0.0
2,2,100,COCONUT_COUPON,681.0,19.0,680.0,20.0,NaN,NaN,682,39,NaN,NaN,NaN,NaN,681.5,0.0
3,2,100,COCONUT,10134.0,150.0,NaN,NaN,NaN,NaN,10136,106,10137.0,44.0,NaN,NaN,10135.0,0.0
4,2,200,COCONUT,10135.0,136.0,10134.0,55.0,NaN,NaN,10136,136,10137.0,55.0,NaN,NaN,10135.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2,999700,COCONUT,9986.0,143.0,9985.0,51.0,NaN,NaN,9987,143,9988.0,51.0,NaN,NaN,9986.5,0.0
19996,2,999800,COCONUT_COUPON,620.0,44.0,NaN,NaN,NaN,NaN,621,44,NaN,NaN,NaN,NaN,620.5,0.0
19997,2,999800,COCONUT,9989.0,119.0,9988.0,57.0,NaN,NaN,9990,119,9991.0,57.0,NaN,NaN,9989.5,0.0
19998,2,999900,COCONUT,9989.0,212.0,NaN,NaN,NaN,NaN,9991,137,9992.0,75.0,NaN,NaN,9990.0,0.0


In [12]:
def get_popular_price(row: Any, bid_ask: str) -> int:
    best_price = -1
    max_volume = -1

    for i in range(1, 4):
        volume = getattr(row, f"{bid_ask}_volume_{i}")
        if math.isnan(volume):
            break

        if volume > max_volume:
            best_price = getattr(row, f"{bid_ask}_price_{i}")
            max_volume = volume

    return best_price

def get_product_prices(prices: pd.DataFrame, product: str) -> np.ndarray:
    prices = prices[prices["product"] == product]

    mid_prices = []
    for row in prices.itertuples():
        popular_buy_price = get_popular_price(row, "bid")
        popular_sell_price = get_popular_price(row, "ask")
        mid_prices.append((popular_buy_price + popular_sell_price) / 2)

    return np.array(mid_prices)

for day in range(1, 4):
    prices = get_prices(4, day)

    cocos = get_product_prices(prices, "COCONUT")
    coups = get_product_prices(prices, "COCONUT_COUPON")

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(y=cocos, name="Price"), secondary_y=False)
    # fig.add_trace(go.Scatter(y=buy_signal.astype(int), name="Buy"), secondary_y=True)
    # fig.add_trace(go.Scatter(y=sell_signal.astype(int), name="Sell"), secondary_y=True)

    # for premium in range(200, 801, 100):
    #     signal = gift_basket_contents + premium
    #     fig.add_trace(go.Scatter(y=signal, name=f"Signal - {premium}"), secondary_y=False)

    fig.add_trace(go.Scatter(y=(cocos - 10000) + coups, name="Signal"), secondary_y=True)

    fig.update_layout(title_text=f"Day {day}")
    fig.show()